In [1]:
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
def init_browser():
    executable_path = {'executable_path': ChromeDriverManager().install()}
    return Browser('chrome', **executable_path, headless=False)

In [2]:
# Cities to search
cities = ['Los-Angeles_CA','New-York_NY', 'Chicago_IL']
# , 'Houston_TX', 'Phoenix_AZ', 'Philadelphia_PA', 'San-Antonio_TX', 'San-Diego_CA', 'Dallas_TX', 'San-Jose_CA']

cities =['Los-Angeles_CA']

In [3]:
# Start browser
browser = init_browser()



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\kesam\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


In [4]:
# Create blank lists to store attributes
prices = []
beds = []
baths = []
sizes = []
addresses = []
statuses = []
pages = []
types = []
years = []

In [6]:
# Loop through each city
for city in cities:

    # Loop through each search result page
    for i in range(1, 5, 1):

        # Set variables
        url = f"https://www.realtor.com/realestateandhomes-search/{city}/pg-{i}"

        time.sleep(5) 
        browser.visit(url)
        html = browser.html
        soup = bs(html, "html.parser")

        # Identify all listings
        listings = soup.find_all('li', attrs={"data-testid": "result-card"})

        # Loop through each listing
        for listing in listings:
            price = listing.find('span', attrs={"data-label": "pc-price"})
            bed = listing.find('li', attrs={"data-label": "pc-meta-beds"})
            bath = listing.find('li', attrs={"data-label": "pc-meta-baths"})
            size = listing.find('li', attrs={"data-label": "pc-meta-sqft"})
            address = listing.find('div', attrs={"data-label": "pc-address"})
            status = listing.find('span', attrs={"class": "jsx-3853574337 statusText"})

            if price and price.text:
                prices.append(price.text)
            else: 
                prices.append('No Info')

            if bed and bed.text:
                beds.append(bed.text)
            else: 
                beds.append('No Info')

            if bath and bath.text:
                baths.append(bath.text)
            else: 
                baths.append('No Info')

            if size and size.text:
                sizes.append(size.text)
            else: 
                sizes.append('No Info')

            if address and address.text:
                addresses.append(address.text)
            else: 
                addresses.append('No Info')

            if status and status.text:
                statuses.append(status.text)
            else: 
                statuses.append('No Info')

                
            # Identify URL to listing detail page
            page = listing.find('a').get('href')

            time.sleep(5)
            browser.click_link_by_href(page)

            html = browser.html
            detail_page = bs(html, "html.parser")
#             print(detail_page.prettify())
            # Extract property type and year property was built
            details = detail_page.find_all('li', attrs={"class": "rui-patterns__sc-2lxyoa-0 ivcDnD"})

            property_type = details[0].find('div', attrs={"class": "rui__ygf76n-0 gFfdWq rui__sc-17fo6pt-0 iRbBbO"})
#             year_built = details[4].find('div', attrs={"class": "rui__ygf76n-0 gFfdWq rui__sc-17fo6pt-0 iRbBbO"})

            if property_type and property_type.text:
                types.append(property_type.text)
            else: 
                types.append('No Info')
#             if year_built and year_built.text:
#                 years.append(year_built.text)
#             else: 
#                 years.append('No Info')

            time.sleep(5)    
            browser.back()

In [ ]:
listings_df = pd.DataFrame({'Address': addresses, 'Status': status, 'Property Type': types, 'Year Built': years, 'Price': prices, 'Bed': beds, 'Bath': baths, 'Size': sizes})
# df.drop_duplicates(subset='Address', keep="last")
listings_df.to_csv('Rental_Listings.csv', index=False, encoding='utf-8')
listings_df

In [ ]:
browser.quit()

In [7]:
import psycopg2
from sqlalchemy import create_engine
from Config import driver, username, password, host, port, database

connection_string = f"{driver}://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)
connection = engine.connect()

In [ ]:
listings_df.to_sql('listings',connection, if_exists='append', index=False)

In [ ]:
listings_df.head()

In [ ]:
prices